In [19]:
#The following command resets the environment
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

#!pip install pymongo

In [28]:
from pymongo import MongoClient
import pprint
import pymongo
import pandas as pd
import datetime
from bson.code import Code

In [30]:
#Creates a connection to the MongoDB server running on localhost
#MongoClient object allows you to interact with MongoDB databases and collections.
#Drop Movies if exists, to clear the database
#Throws error if fail
try:
    client = MongoClient("mongodb://localhost:27017/")
    client.drop_database("Movies")
    client.drop_database("Test")
except Exception as e:
    print(e)

In [32]:
#Create test database
try:
    db = client.get_database("Test")
    print("Database 'Test' accessed or created successfully.")
    db.Test.insert_one({"name": "Sample Data"})
except Exception as e:
    print(f"An error occurred: {e}")

#Data insertions and table creations are verified in MongoDB Compass

Database 'Test' accessed or created successfully.


In [34]:
#Create assignment database
try:
    db = client.get_database("Movies")
    print("Database 'Movies' accessed or created successfully.")
except Exception as e:
    print(f"An error occurred: {e}")

Database 'Movies' accessed or created successfully.


In [36]:
#Read CSV files
movies = pd.read_csv("movies.csv")

#Extract year from titles and create a new column for the movies collection to make aggregation easier
#create a new column, initialize to null
movies["year"] = None
for i in range(len(movies)):
    temp = movies.loc[i,"title"]
    #Validate proper title format
    if(temp[-6] == '(' and temp[-1] == ')'):
        #insert year into new column, using .loc, now we can use negative indices from -5 to -2, where -6 is ( and -1 ), but the substring 
        #stops before -1
        movies.loc[i, "year"] = temp[-5:-1]
    else:
        pass
    
ratings = pd.read_csv("ratings.csv")
tags = pd.read_csv("tags.csv")

#Create collections IF NOT EXISTS
if "movies" not in db.list_collection_names():
    db.create_collection("movies")
if "ratings" not in db.list_collection_names():
    db.create_collection("ratings")
if "tags" not in db.list_collection_names():
    db.create_collection("tags")

#Insert records into 3 collections, a record refers to a single row of data in a collection(table)
db.movies.insert_many(movies.to_dict('records'))
db.ratings.insert_many(ratings.to_dict('records'))
db.tags.insert_many(tags.to_dict('records'))


InsertManyResult([ObjectId('6751d8ef725bab08a5665f11'), ObjectId('6751d8ef725bab08a5665f12'), ObjectId('6751d8ef725bab08a5665f13'), ObjectId('6751d8ef725bab08a5665f14'), ObjectId('6751d8ef725bab08a5665f15'), ObjectId('6751d8ef725bab08a5665f16'), ObjectId('6751d8ef725bab08a5665f17'), ObjectId('6751d8ef725bab08a5665f18'), ObjectId('6751d8ef725bab08a5665f19'), ObjectId('6751d8ef725bab08a5665f1a'), ObjectId('6751d8ef725bab08a5665f1b'), ObjectId('6751d8ef725bab08a5665f1c'), ObjectId('6751d8ef725bab08a5665f1d'), ObjectId('6751d8ef725bab08a5665f1e'), ObjectId('6751d8ef725bab08a5665f1f'), ObjectId('6751d8ef725bab08a5665f20'), ObjectId('6751d8ef725bab08a5665f21'), ObjectId('6751d8ef725bab08a5665f22'), ObjectId('6751d8ef725bab08a5665f23'), ObjectId('6751d8ef725bab08a5665f24'), ObjectId('6751d8ef725bab08a5665f25'), ObjectId('6751d8ef725bab08a5665f26'), ObjectId('6751d8ef725bab08a5665f27'), ObjectId('6751d8ef725bab08a5665f28'), ObjectId('6751d8ef725bab08a5665f29'), ObjectId('6751d8ef725bab08a5665f

In [37]:
#Insert 5 custom movies
db.movies.insert_one({"movieId": 200000, "title": "SomeMovie1", "genres": "Action|Sci-Fi", "year": '2000'})
db.movies.insert_one({"movieId": 200001, "title": "SomeMovie2", "genres": "Action|Sci-Fi", "year": '2001'})
db.movies.insert_one({"movieId": 200002, "title": "SomeMovie3", "genres": "Adventure|Drama|Sci-Fi", "year": '2002'})
db.movies.insert_one({"movieId": 200003, "title": "SomeMovie4", "genres": "Thriller|Drama", "year": '2003'})
db.movies.insert_one({"movieId": 200004, "title": "SomeMovie5", "genres": "Action|Crime|Drama", "year": '2004'})

#Insert ratings
my_ratings = [
    {"userId": 1234567, "movieId": 200000, "rating": 5.0},
    {"userId": 1234567, "movieId": 200001, "rating": 4.5},
    {"userId": 1234567, "movieId": 200002, "rating": 4.8},
    {"userId": 1234567, "movieId": 200003, "rating": 5.0},
    {"userId": 1234567, "movieId": 200004, "rating": 4.9}
]
db.ratings.insert_many(my_ratings)

#Insert tags
my_tags = [
    {"userId": 1234567, "movieId": 200000, "tag": "Tag1"},
    {"userId": 1234567, "movieId": 200001, "tag": "Tag2"},
    {"userId": 1234567, "movieId": 200002, "tag": "Tag3"},
    {"userId": 1234567, "movieId": 200003, "tag": "Tag4"},
    {"userId": 1234567, "movieId": 200004, "tag": "Tag5"}
]
db.tags.insert_many(my_tags)


InsertManyResult([ObjectId('6751d8ef725bab08a5666d7e'), ObjectId('6751d8ef725bab08a5666d7f'), ObjectId('6751d8ef725bab08a5666d80'), ObjectId('6751d8ef725bab08a5666d81'), ObjectId('6751d8ef725bab08a5666d82')], acknowledged=True)

In [38]:
#Query using Aggregation Pipeline

#1. Develop code to find number of movies released per year. 
movies_per_year = db.movies.aggregate([
    {
        # For the records in the collection with an invalid title/year format, exclude from the aggregation
        "$match": {
            #year not equal to null
            "year": {"$ne": None}
        }
    },
    {
        "$group":{
            #The year of a movie is not explicity stored as its own attribute in the movies collection, therefore we need to extract it from the end of
            #each title string, which is always the last 6 characters of the title for each record. This can be achived using the $substr which returns
            #a substring of a string using an index range parameters
            "_id": "$year",
                #Last characters 2,3,4,5 of the last 6 characters of title, therefore start = -6, length = 4, where -6 is '('. 
                #We want 4 characters from the start for a year xxxx, therefore length=4, starting from the first index after -6
                #mongoDB does not actually support negative indicies, therefore we use "$subtract" to get the starting index
                #Scrap the above method, just extract the title using python from the csv file and create a new attribute in the collection after
                #establishing a connection.
            #Now for each year, count the number of movies in the collection, done using $sum, which we increment for each entry found. 
             #"$sum:x" where x is the amount we increment by for each appearance
            "Released this year": {"$sum":1}
        }
    },
    {
        #Sort by year in ascending order
        "$sort":{
            "_id":-1
        }
    }
])

for i in movies_per_year:
    print(i)

{'_id': '2018', 'Released this year': 41}
{'_id': '2017', 'Released this year': 147}
{'_id': '2016', 'Released this year': 218}
{'_id': '2015', 'Released this year': 274}
{'_id': '2014', 'Released this year': 277}
{'_id': '2013', 'Released this year': 239}
{'_id': '2012', 'Released this year': 232}
{'_id': '2011', 'Released this year': 252}
{'_id': '2010', 'Released this year': 247}
{'_id': '2009', 'Released this year': 282}
{'_id': '2008', 'Released this year': 268}
{'_id': '2007', 'Released this year': 282}
{'_id': '2006', 'Released this year': 295}
{'_id': '2005', 'Released this year': 273}
{'_id': '2004', 'Released this year': 280}
{'_id': '2003', 'Released this year': 280}
{'_id': '2002', 'Released this year': 312}
{'_id': '2001', 'Released this year': 295}
{'_id': '2000', 'Released this year': 284}
{'_id': '1999', 'Released this year': 262}
{'_id': '1998', 'Released this year': 257}
{'_id': '1997', 'Released this year': 260}
{'_id': '1996', 'Released this year': 276}
{'_id': '199

In [39]:
#2. Develop code to find number of movies per genre. 
movies_per_genre = db.movies.aggregate([
    {
        #Transform the genres attribute by splitting its genres string into separate elements in an array using $split via the "|" delimiter 
        "$project":{
            "genreArray": {"$split": ["$genres", "|"]}
        }
    },
    {
        #From the array of genre elements, unwind the create arrays by generating separate entries for each genre in the array calculated in the
        #$project stage. The result is then used as the input for the next pipeline stage.
        "$unwind": "$genreArray"
    },
    {
        "$group":{
            #Group by individual genres, count movies of each genre
            "_id": "$genreArray",
            "Movie count":{"$sum":1}
        }
    },
    {
        #Sort by total count for each genre, descending
        "$sort":{
            "Movie count":-1
        }
    },
])

for i in movies_per_genre:
    print(i)


{'_id': 'Drama', 'Movie count': 4364}
{'_id': 'Comedy', 'Movie count': 3756}
{'_id': 'Thriller', 'Movie count': 1895}
{'_id': 'Action', 'Movie count': 1831}
{'_id': 'Romance', 'Movie count': 1596}
{'_id': 'Adventure', 'Movie count': 1264}
{'_id': 'Crime', 'Movie count': 1200}
{'_id': 'Sci-Fi', 'Movie count': 983}
{'_id': 'Horror', 'Movie count': 978}
{'_id': 'Fantasy', 'Movie count': 779}
{'_id': 'Children', 'Movie count': 664}
{'_id': 'Animation', 'Movie count': 611}
{'_id': 'Mystery', 'Movie count': 573}
{'_id': 'Documentary', 'Movie count': 440}
{'_id': 'War', 'Movie count': 382}
{'_id': 'Musical', 'Movie count': 334}
{'_id': 'Western', 'Movie count': 167}
{'_id': 'IMAX', 'Movie count': 158}
{'_id': 'Film-Noir', 'Movie count': 87}
{'_id': '(no genres listed)', 'Movie count': 34}


In [40]:
#3. Develop code to find number of movies per rating. 
movies_per_rating = db.ratings.aggregate([
    {
        "$group":{
            #Group by rating, count # of movies for each rating
            "_id": "$rating",
            "Movie count":{"$sum":1}
        }
    },
    {
        #Sort by total count of movies for each rating 0-5
        "$sort":{
            "Movie count":-1
        }
    },
])

for i in movies_per_rating:
    print(i)


{'_id': 4.0, 'Movie count': 26818}
{'_id': 3.0, 'Movie count': 20047}
{'_id': 5.0, 'Movie count': 13213}
{'_id': 3.5, 'Movie count': 13136}
{'_id': 4.5, 'Movie count': 8552}
{'_id': 2.0, 'Movie count': 7551}
{'_id': 2.5, 'Movie count': 5550}
{'_id': 1.0, 'Movie count': 2811}
{'_id': 1.5, 'Movie count': 1791}
{'_id': 0.5, 'Movie count': 1370}
{'_id': 4.8, 'Movie count': 1}
{'_id': 4.9, 'Movie count': 1}


In [41]:
#4. Develop code to find number of movies tagged.  
count_movies_tagged = db.tags.aggregate([
    {
        "$group":{
            #Group by movieId to eliminate and duplicate movieIds from the collection of tagged movies
            "_id": "$movieId"
        }
    },
    {
        "$group":{
            #Group every record of unique movieIds into one category, then count the total number of tagged movies
            "_id": None,
            "Count of tagged movies": {"$sum":1}
        }
    }
])

for i in count_movies_tagged:
    print(i)

{'_id': None, 'Count of tagged movies': 1577}


In [42]:
#5. Develop code to find the most popular tag. 
most_popular_tag = db.tags.aggregate([
    {
        "$group":{
            #Group by tag to fidn distinct tags and count the number of times the tag was used for a movie
            "_id": "$tag",
            "Tag count":{"$sum":1}
        }
    },
        {
        #Sort by total count of a tag appearing for tagged movies
        "$sort":{
            "Tag count":-1
        }
    },
    {
        "$limit":1
    }
])

print(next(most_popular_tag))

{'_id': 'In Netflix queue', 'Tag count': 131}
